In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import random
import time
from helper_functions import *
import csv

In [4]:
urls = {
    'Towards Data Science': 'https://towardsdatascience.com/archive/{0}/{1:02d}/{2:02d}'
}

In [5]:
all_days = range(1, 366)

In [11]:
#IMPORTANT: This code overwrites what's in the file so only run it ONCE to create the file
#with proper headings
cols=['url', 'title', 'subtitle',
    'n_sections', 'n_paragraphs', 'section_titles', 'story_text',
    'claps', 'responses',
    'reading_time', 'publication', 'date','year']

filename = "all_articles_2021_2022.csv"

years = [2021,2022]

with open(filename, mode="w") as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(cols)
                    
file.close()  

In [12]:
data = []
article_id = 0
years = years
i = 0
n = len(all_days)
for year in years:
    for d in all_days:
        i += 1
        month, day = convert_day(d)
        date = '{0}-{1:02d}-{2:02d}'.format(year, month, day)
        print(f'{i} / {n} ; {date}',end='\r')
        for publication, url in urls.items():
            rand_sleep() #sleep random number of seconds
            response = requests.get(url.format(year, month, day), allow_redirects=True)
            if not response.url.startswith(url.format(year, month, day)):
                continue
            page = response.content
            soup = BeautifulSoup(page, 'html.parser')
            articles = soup.find_all(
                "div",
                class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
            for article in articles:
                title = article.find("h3", class_="graf--title")
                if title is None:
                    continue
                title = title.contents[0]
                article_id += 1
                subtitle = article.find("h4", class_="graf--subtitle")
                subtitle = subtitle.contents[0] if subtitle is not None else ''
                #image = article.find("img", class_="graf-image")
                #image = '' if image is None else get_img(image['src'], 'images', f'{article_id}')
                article_url = article.find_all("a")[3]['href'].split('?')[0]
                number_sections, number_paragraphs, section_titles, story_text = get_article_text(article_url)
                buttons = article.find_all("button")
                claps = get_claps(buttons[1].contents[0]) if len(buttons) > 0 else None
                reading_time = article.find("span", class_="readingTime")
                reading_time = 0 if reading_time is None else int(reading_time['title'].split(' ')[0])
                responses = article.find_all("a")
                if len(responses) == 7:
                    responses = responses[6].contents[0].split(' ')
                    if len(responses) == 0:
                        responses = 0
                    else:
                        responses = responses[0]
                else:
                    responses = 0

                info = [article_url, title, subtitle,\
                             number_sections, number_paragraphs, section_titles, story_text,\
                             claps, responses,\
                             reading_time, publication, date,year]
                
                with open(filename, mode="a") as file:
                    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    writer.writerow(info)
                    
                file.close()    


ValueError: sleep length must be non-negative